In [34]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as graph
import math
from sklearn import mixture
import warnings
warnings.filterwarnings("ignore")


In [27]:
class Gaussian_mixture_model:
    def __init__(self):
        self.n_clusters = None
    # data as a matrix format
    def data_matrix(self,data):
        X = []
        for i in range(len(data)):
            X.append(np.array(data.iloc[i]))
        return (np.matrix(X))

    # defining the mean
    def mean(self,prob_mat,data):
        X = self.data_matrix(data)
        means = []
        for i in range(len(prob_mat)):
            x = 0
            y = 0
            for j in range(len(prob_mat[0])):
                x += prob_mat[i][j] * X[j].T
                y += prob_mat[i][j]
            means.append(x/y)
        return means

    def covariance(self,prob_mat,data):
        X = self.data_matrix(data)
        cov = []
        means = self.mean(prob_mat,data)
        for i in range(len(prob_mat)):
            x = 0
            y = 0
            for j in range(len(prob_mat[0])):
                x += prob_mat[i][j] * np.matmul((X[j].T - means[i]),(X[j].T - means[i]).T)
                y += prob_mat[i][j]
            cov.append(x/(y-1))
        return cov

    
    #defining multivariate gaussian distribution
    def multi_gauss_dis(self,prob_mat , data):
        means = self.mean(prob_mat , data)
        cov = self.covariance(prob_mat , data)
        X = self.data_matrix(data)
        Q = np.empty([len(prob_mat) , len(prob_mat[0])])
        for i in range(len(prob_mat)):
            for j in range(len(prob_mat[0])):
                Q[i][j]  = float(1/(2 * math.pi * np.linalg.det(cov[i])*np.matmul((X[j] - means[i].T),np.matmul(np.linalg.inv(cov[i]),(X[j].T - means[i])))))
        return Q


    #defining probability of each class
    def class_probability(self,prob_mat , data):
        class_prob = []
        for i in range(len(prob_mat)):
            x = 0
            for j in range(len(prob_mat[0])):
                x += prob_mat[i][j]
            class_prob.append(x/len(data))
        return class_prob


    # allocate the class
    def calculate_class(self,data , prob_mat):
        t = np.zeros([len(prob_mat),len(prob_mat[0])])
        X = self.data_matrix(data)
        Q = self.multi_gauss_dis(prob_mat , data)
        prob_class = self.class_probability(prob_mat , data)
        for i in range(len(Q)):
            for j in range(len(Q[0])):
                Q[i][j] = prob_class[i] * Q[i][j]
        x = []
        for i in range(len(Q[0])):
            x.append(np.argmax(Q[:,i]))
        j = 0
        for i in range(len(prob_mat[0])):
            t[:,i][x[j]] = 1.0
            j +=1
        return t



    def fit(self,prob_mat , data):
        for i in range(100):
            # E step
            prob_mat = self.calculate_class(data , prob_mat)
        return prob_mat



    def cluster_alloc(self,prob_mat,data):
        X = self.data_matrix(data)
        c1 = []
        c2 = []
        c3 = []
        c4 = []
        c5 = []
        for i in range(len(data)):
            if prob_mat[:,i][0] == 1:
                c1.append(X[i])
            if prob_mat[:,i][1] == 1:
                c2.append(X[i])
            if prob_mat[:,i][2] == 1:
                c3.append(X[i])
            if prob_mat[:,i][3] == 1:
                c4.append(X[i])
            if prob_mat[:,i][4] == 1:
                c5.append(X[i])
        return c1,c2,c3,c4,c5


In [58]:
prob_mat = [[0,0,0,1,0]*100,[1,0,0,0,0]*100,[0,1,0,0,0]*100,[0,0,1,0,0]*100,[0,0,0,0,1]*100]
X = gmm.data_matrix(data)

In [30]:

data = pd.read_table("ECG5000_TRAIN.tsv",sep='\t')
data.to_csv('ECG5000_TRAIN.csv',index=False)
data = pd.read_csv("ECG5000_TRAIN.csv", names = ['x' + str(i+1) for i in range(len(data.columns))])
data = data.drop('x1',axis = 1)
# data =data.drop(0,axis = 0)


,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,...,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.160348,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.560327,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,1.284825,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.491173,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,0.966606,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.478577,-1.779959,-2.398159,-3.170112,-3.559732,-3.573956,-2.989770,-2.270605,-1.688277,-1.359872,...,1.294840,1.160885,1.456331,2.209421,2.507175,2.198534,1.705849,1.492642,1.561890,1.520161
496,-1.325210,-2.480992,-2.965356,-3.342392,-3.176351,-2.891528,-2.369679,-1.598750,-1.071751,-0.891843,...,0.552076,-0.172154,-0.864803,-1.549854,-2.460243,-3.366562,-3.466546,-2.718380,-1.855209,-1.539958
497,-0.021964,-0.912434,-1.903353,-2.662829,-3.122156,-3.451490,-3.392982,-2.929937,-2.256294,-1.690706,...,1.528745,1.339479,1.457995,2.128078,2.630759,2.295748,1.764967,1.444280,1.432347,1.457028
498,0.288011,-1.098020,-2.500250,-3.598599,-3.650608,-3.281587,-2.231601,-1.250656,-1.072574,-0.434310,...,0.081634,-0.029242,0.071414,0.118161,-0.071967,-0.171214,0.131211,0.049872,0.010915,-0.081534


In [57]:
for i in data.columns:
    data[i] = (data[i] - np.min(data[i]))/(np.max(data[i])-np.min(data[i]))
data

,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,...,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141
0,0.462013,0.317872,0.199376,0.109662,0.156437,0.279027,0.375475,0.321855,0.409652,0.534643,...,0.535800,0.706285,0.734077,0.703096,0.713261,0.675798,0.651231,0.671160,0.797764,0.692388
1,0.334522,0.155546,0.123005,0.085474,0.211781,0.321488,0.505317,0.534780,0.532296,0.659773,...,0.602096,0.665101,0.692965,0.701767,0.733581,0.713044,0.682927,0.757034,0.821380,0.504070
2,0.403377,0.350313,0.184409,0.073519,0.185949,0.336118,0.468004,0.406307,0.426209,0.554687,...,0.722181,0.721522,0.674326,0.634944,0.629349,0.554403,0.554162,0.697259,0.795205,0.621316
3,0.539795,0.444553,0.222877,0.114432,0.173342,0.207360,0.204146,0.359787,0.389032,0.417139,...,0.590634,0.634671,0.669520,0.675501,0.750072,0.762567,0.748527,0.798358,0.855818,0.625735
4,0.579752,0.588910,0.406606,0.167723,0.162355,0.221246,0.300960,0.330841,0.324583,0.468275,...,0.669437,0.764165,0.737776,0.739878,0.824073,0.803221,0.744719,0.783071,0.881506,0.834351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.414794,0.463213,0.404607,0.291599,0.284179,0.261562,0.204886,0.205261,0.301375,0.322236,...,0.723841,0.766112,0.811764,0.901337,0.982732,0.918332,0.839907,0.851943,0.875125,0.845759
496,0.305584,0.365921,0.319994,0.265028,0.344174,0.381988,0.335744,0.378445,0.453870,0.434900,...,0.600729,0.549815,0.466841,0.373718,0.288649,0.222882,0.179514,0.295942,0.459871,0.492084
497,0.473694,0.583611,0.478422,0.369838,0.352655,0.283173,0.119797,0.035305,0.160878,0.242598,...,0.762610,0.795090,0.812011,0.889920,1.000000,0.930481,0.847455,0.845558,0.859383,0.838463
498,0.513679,0.557855,0.389378,0.225513,0.269958,0.313155,0.364882,0.468173,0.453666,0.545038,...,0.522754,0.573004,0.605964,0.607826,0.622356,0.622194,0.638863,0.661448,0.686647,0.660642


In [59]:
x = gmm.multi_gauss_dis(prob_mat,data)
y = gmm.covariance(prob_mat,data)
np.linalg.det(y[1])

0.0

In [31]:
gmm = Gaussian_mixture_model()
prob_mat = gmm.fit(prob_mat,data)


In [32]:
c1,c2,c3,c4,c5 = gmm.cluster_alloc(prob_mat,data)

In [33]:
print(np.shape(c1))
print(np.shape(c2))
print(np.shape(c3))
print(np.shape(c4))
print(np.shape(c5))

(500, 1, 140)
(0,)
(0,)
(0,)
(0,)


In [38]:
#training the model
gm = mixture.GaussianMixture(n_components = 5)
gm.fit(np.array(X))

GaussianMixture(n_components=5)

In [39]:
#clusterring the data
y_pred = gm.predict(np.array(X))

In [40]:
c = list(set(y_pred))
c

[0, 1, 2, 3, 4]

In [41]:
c1 = X[y_pred == 0]
c2 = X[y_pred == 1]
c3 = X[y_pred == 2]
c4 = X[y_pred == 3]
c5 = X[y_pred == 4]

In [43]:
np.shape(c1)

(20, 140)